In [1]:
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv("./magic04.data", names=["fLength","fWidth","fSize","fConc","fConc1",'fAsym','fM3Long','fM3Trans','fAlpha','fDist',"class",])
df.head(-5)

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
...,...,...,...,...,...,...,...,...,...,...,...
19010,32.4902,10.6723,2.4742,0.4664,0.2735,-27.0097,-21.1687,8.4813,69.1730,120.6680,h
19011,79.5528,44.9929,3.5488,0.1656,0.0900,-39.6213,53.7866,-30.0054,15.8075,311.5680,h
19012,31.8373,13.8734,2.8251,0.4169,0.1988,-16.4919,-27.1448,11.1098,11.3663,100.0566,h
19013,182.5003,76.5568,3.6872,0.1123,0.0666,192.2675,93.0302,-62.6192,82.1691,283.4731,h


In [3]:
df['class'] = (df['class'] == 'g').astype(int)
df['class'].unique()

array([1, 0])

In [4]:
X = df.iloc[:, :-7]
y = df['class']
X.columns, y.unique()

(Index(['fLength', 'fWidth', 'fSize', 'fConc'], dtype='object'), array([1, 0]))

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)
X_train.count(), X_test.count()

(fLength    15216
 fWidth     15216
 fSize      15216
 fConc      15216
 dtype: int64,
 fLength    3804
 fWidth     3804
 fSize      3804
 fConc      3804
 dtype: int64)

In [7]:
y_train.value_counts()

1    9871
0    5345
Name: class, dtype: int64

In [8]:
X_train.isnull().sum()

fLength    0
fWidth     0
fSize      0
fConc      0
dtype: int64

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [10]:
numerical_cols = [col for col in X.columns if X[col].dtype.name == 'float64']
cat_col = [col for col in df.columns if df[col].dtype.name=='object']
numerical_cols, cat_col

(['fLength', 'fWidth', 'fSize', 'fConc'], [])

In [11]:
numerical_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler()),
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols)
    ]
)

preprocess

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scale', StandardScaler())]),
                                 ['fLength', 'fWidth', 'fSize', 'fConc'])])

In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)

In [13]:
from sklearn.metrics import classification_report

my_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', model)
])
my_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['fLength', 'fWidth', 'fSize',
                                                   'fConc'])])),
                ('model', RandomForestClassifier(random_state=0))])

In [14]:
my_pipeline.fit(X_train, y_train)

preds = my_pipeline.predict(X_test)

print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.79      0.65      0.72      1343
           1       0.83      0.91      0.87      2461

    accuracy                           0.82      3804
   macro avg       0.81      0.78      0.79      3804
weighted avg       0.82      0.82      0.81      3804



In [15]:
preds, y_test

(array([1, 1, 1, ..., 0, 0, 1]),
 5978     1
 1891     1
 9547     1
 18885    0
 16665    0
         ..
 12020    1
 463      1
 15564    0
 13840    0
 2334     1
 Name: class, Length: 3804, dtype: int32)

In [16]:
import pickle


pickle.dump(my_pipeline, open('model.pkl', 'wb'))

In [17]:
model = pickle.load(open('D:/kalpana/app/model/model.pkl', 'rb'))

In [18]:
features = pd.DataFrame([[1,2,3,4]], columns=['fLength', 'fWidth', 'fSize', 'fConc'])
model.predict(features)

array([0])

In [19]:
type (X_test)

pandas.core.frame.DataFrame

In [20]:
diction = {
    'flength': 1,
    'fwidth': 2,
    'fSize': 3,
    'fConc': 4
}

In [21]:
features = list(diction.values())
features

[1, 2, 3, 4]

In [22]:
import joblib

ref_cols = list(X.columns)
target = 'class'

In [23]:
joblib.dump([my_pipeline, ref_cols, target], filename='model.pkl')

['model.pkl']

In [24]:
model, ref_cols, target = joblib.load('model.pkl')

In [26]:
features = pd.DataFrame([[1,2,3,4]], columns=['fLength', 'fWidth', 'fSize', 'fConc'])
model.predict(features)[0]

0